In [1]:
import csv, collections, os
import pandas as pd

In [2]:
train_dir = '../../data/.train/.task148/data/train/images/'
train_csv = '../../data/.train/.task148/data/train/train.csv'
train_ccsv = '../../data/.train/.task148/data/train/train_custom.csv'

In [3]:
test_dir = '../../data/.train/.task148/data/test/images/'
test_csv = '../../data/.train/.task148/data/test/test.csv'

In [4]:
features = []
with open(train_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i == 0:
            for j in range(len(line)):
                print(j, line[j])
                features.append(line[j])

0 ID
1 위도,경도
2 도로명 기본 주소
3 도로명 번지
4 확장 도로명 참고 주소
5 표준 도로명 참고 주소
6 지번 주소
7 지번 주소 번호
8 지번 주소명
9 시도명
10 구역번호(우편번호)
11 구우편번호(6자리)
12 법정동코드
13 행정동코드
14 도로명 코드
15 짧은 시도
16 시군구명
17 읍면동 이름
18 법정동명
19 행정동명
20 도로명
21 도로명 주번호
22 도로명 부번호
23 지번 주번지
24 지번 부번지
25 리 명
26 대표 건물관리번호(25)
27 건축물대장번호
28 건물용도
29 공동주택 여부
30 리 여부
31 산 여부
32 지하 여부
33 대장종류코드번호
34 동명
35 주부속구분코드
36 주부속구분코드명
37 건축물대지면적
38 건축면적
39 건축물건폐율
40 건축물연면적
41 용적율산정연면적
42 건축물용적율
43 구조코드번호
44 구조코드명
45 기타구조코드명
46 주용도코드번호
47 주용도코드명
48 기타용도코드명
49 지붕코드번호
50 지붕코드명
51 기타지붕코드명
52 세대수
53 가구수
54 건물높이
55 건축물지상층수
56 건축물지하층수
57 충동연면적
58 옥내기계식대수
59 옥내기계식면적
60 옥외기계식대수
61 옥외기계식면적
62 옥내자주식대수
63 옥내자주식면적
64 옥외자주식대수
65 옥외자주식면적
66 사용승인일자
67 Target


In [5]:
features_dic = {}
idxs = [29, 31, 32, 36, 44, 50]

with open(train_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i != 0:
            for idx in idxs:
                if idx not in features_dic: features_dic[idx] = set()
                features_dic[idx].add(line[idx])
                
for feature in features_dic:
    print(feature, len(sorted(features_dic[feature])), sorted(features_dic[feature]))

29 3 ['0', '1', '2']
31 2 ['0', '1']
32 2 ['0', '1']
36 3 ['', '부속건축물', '주건축물']
44 28 ['', ' ', '강파이프구조', '경량철골구조', '공업화박판강구조(PEB)', '기타강구조', '기타구조', '기타조적구조', '기타철골철근콘크리트구조', '기타콘크리트구조', '목구조', '벽돌구조', '블록구조', '석구조', '시멘트블럭조', '일반목구조', '일반철골구조', '조립식판넬조', '조적구조', '철골구조', '철골철근콘크리트구조', '철골철근콘크리트합성구조', '철골콘크리트구조', '철근콘크리트구조', '콘크리트구조', '통나무구조', '트러스구조', '프리케스트콘크리트구조']
50 6 ['', ' ', '(철근)콘크리트', '기와', '기타지붕', '슬레이트']


In [6]:
temp_dic = {}
label_5 = {}
label_3 = {}

with open(train_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i != 0:
            target = line[67]
            if ((line[29] == '0') and (line[31] == '0') and (line[32] == '0') and (line[36] != '') 
                and (line[44] in ['경량철골구조', '벽돌구조', '일반목구조', '일반철골구조', '철근콘크리트구조']) 
                and (line[50] in ['(철근)콘크리트', '기와', '기타지붕'])):
            
                if target not in label_5: label_5[target] = 0
                label_5[target] += 1
            elif i != 0:
                target = line[67]
                if target not in label_3: label_3[target] = 0
                label_3[target] += 1

In [7]:
print(collections.Counter(label_3))
print(collections.Counter(label_5))

Counter({'10_콘크리트외벽': 2276, '30_판넬외벽': 499, '20_조적외벽': 188})
Counter({'10_콘크리트외벽': 10350, '30_판넬외벽': 4235, '20_조적외벽': 823, '40_유리외벽': 29, '50_기타외벽': 29})


In [8]:
with open(test_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i != 0:
            for idx in idxs:
                if line[idx] not in features_dic[idx]:
                    print(idx, line[idx])

In [9]:
distributions = {}

with open(train_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i != 0:
            ID = tuple([line[idx] for idx in idxs])
            label = line[67]
            if ID not in distributions: distributions[ID] = {}
            if label not in distributions[ID]: distributions[ID][label] = 0
            distributions[ID][label] += 1
            
for feature in sorted(distributions):
    print(feature, distributions[feature])

('0', '0', '0', '', '', '') {'10_콘크리트외벽': 496, '30_판넬외벽': 231, '20_조적외벽': 39}
('0', '0', '0', '', '철근콘크리트구조', '(철근)콘크리트') {'10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '', '') {'10_콘크리트외벽': 3, '20_조적외벽': 1}
('0', '0', '0', '부속건축물', ' ', ' ') {'10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '강파이프구조', '(철근)콘크리트') {'10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '강파이프구조', '기타지붕') {'30_판넬외벽': 3, '10_콘크리트외벽': 2}
('0', '0', '0', '부속건축물', '경량철골구조', '(철근)콘크리트') {'10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '경량철골구조', '기타지붕') {'10_콘크리트외벽': 44, '30_판넬외벽': 55, '20_조적외벽': 5}
('0', '0', '0', '부속건축물', '기타강구조', '기타지붕') {'20_조적외벽': 2, '10_콘크리트외벽': 1, '30_판넬외벽': 3}
('0', '0', '0', '부속건축물', '기타구조', '기타지붕') {'30_판넬외벽': 1, '10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '기타조적구조', '(철근)콘크리트') {'30_판넬외벽': 1, '10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '기타조적구조', '기타지붕') {'10_콘크리트외벽': 1}
('0', '0', '0', '부속건축물', '벽돌구조', '(철근)콘크리트') {'20_조적외벽': 5, '10_콘크리트외벽': 12, '30_판넬외벽': 4, '50_기타외벽': 1}
('0', '0', '0', '부속건축물', '벽돌구조', '기타지붕') {'20_조적외벽'